# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [67]:
%matplotlib inline
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import re
import os

In [68]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\ellen\Downloads\cdados_p1\22-2a-cd-p1-grupo_alexiabp\notebooks


Carregando a base de dados com as notícias classificadas manualmente:

In [69]:
path = pathlib.Path(r"..\data")
filename = f'{path}\dados.xlsx'

In [70]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Categoria,Titulo,Descrição,Data,Pagina,Target
0,financas,CORREÇÃO-Dólar caminha para 4ª alta semanal se...,(Corrige no 9º parágrafo para “outubro” no lug...,13/05/2022 12:04,71,1.0
1,financas,"Ouro fecha em alta, após quedas recentes, com ...",O contrato mais líquido do ouro fechou em alta...,03/05/2022 15:16,78,2.0
2,financas,"Bolsas da Europa fecham em queda, sem resoluçã...",As principais bolsas europeias fecharam em que...,10/03/2022 14:26,110,0.0
3,economia,"Aneel aprova reajuste médio de 15,77% nas tari...",A Agência Nacional de Energia Elétrica (Aneel)...,23/08/2022 11:21,57,0.0
4,financas,Dólar reduz queda com fortalecimento do DXY ap...,O dólar reduziu a queda intradia frente ao rea...,18/08/2022 10:33,12,1.0


In [71]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Categoria,Titulo,Descrição,Data,Pagina,Target
0,economia,Atividade econômica do País está acima do espe...,O diretor de Política Econômica do Banco Centr...,11/07/2022 12:57,212,0.0
1,economia,Tire suas dúvidas sobre a vacina contra a varí...,Um lote da vacina contra a varíola dos macacos...,17/08/2022 21:20,73,0.0
2,financas,Taxas futuras de juros recuam com ajustes apes...,As taxas de juros negociadas no mercado futuro...,15/07/2022 11:10,32,0.0
3,economia,Corte de Nova York permite que Harvey Weinstei...,Por Daniel Trotta (Reuters) – A mais alta cort...,25/08/2022 11:00,48,0.0
4,financas,"Bolsas de NY fecham em baixa, com guerra na Uc...",As bolsas de Nova York fecharam em baixa nesta...,03/03/2022 18:26,115,0.0


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação das notícias (Target).

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [92]:
def cleanup(text):
    #import string
    text.replace('[...]','')
    punctuation = '[!-.:?;\/]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [106]:
target_0 = test.loc[test['Target']==0,:]
for texto in target_0['Descrição']:
    novo_texto=cleanup(str(texto))
    lista_0+=''.join(novo_texto)
    
for titulo in target_0['Titulo']:
    novo_titulo=cleanup(str(titulo))
    lista_0+=''.join(novo_titulo)
    
    
lista_0.split()

['O',
 'Banco',
 'da',
 'Inglaterra',
 'BoE',
 'na',
 'sigla',
 'em',
 'inglês',
 'decidiu',
 'elevar',
 'sua',
 'taxa',
 'básica',
 'de',
 'juros',
 'em',
 '25',
 'pontosbase',
 'pela',
 'quinta',
 'vez',
 'consecutiva',
 'a',
 '125',
 'em',
 'meio',
 'à',
 'persistência',
 'da',
 'inflação',
 'alta',
 'no',
 'Reino',
 'Unido',
 'após',
 'concluir',
 'reunião',
 'de',
 'política',
 'monetária',
 'nesta',
 'quintafeira',
 '16',
 'A',
 'decisão',
 'do',
 'BC',
 'inglês',
 'veio',
 'em',
 'linha',
 'com',
 'a',
 'expectativa',
 '[…]O',
 'diretor',
 'de',
 'Política',
 'Econômica',
 'do',
 'Banco',
 'Central',
 'BC',
 'Diogo',
 'Guillen',
 'disse',
 'nesta',
 'segundafeira',
 '11',
 'que',
 'a',
 'atividade',
 'econômica',
 'brasileira',
 'desempenhou',
 'em',
 'nível',
 'acima',
 'do',
 'esperado',
 'no',
 'primeiro',
 'semestre',
 'Ele',
 'participou',
 'de',
 'um',
 'webinar',
 'organizado',
 'pelo',
 'Credit',
 'Suisse',
 'pela',
 'manhã',
 'De',
 'acordo',
 'com',
 'Guillen',
 'o',
 

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**